# 経済関係のデータセット

景気ウォッチャー調査：景気に敏感な職種の人々へのインタビュー結果を集計した「街角景気」<br>
https://www5.cao.go.jp/keizai3/watcher/watcher_menu.html

![Watcher](slides/watcher.png)

- 合計のところの値が、全体の指数 (DI: Diffusion Index)

日経平均株価<br>
https://indexes.nikkei.co.jp/nkave/historical/nikkei_stock_average_monthly_jp.csv
- ダウンロードして data フォルダ以下に置いてください

景気ウォッチャーDIと日経平均株価に相関があるか調べる。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Excelファイルの読み込み
# - skiprows: 先頭行の読み飛ばし
# - usecols: 使用する列の指定
watcher = pd.read_excel('data/watcher5.xls', skiprows=range(0,3), usecols='B:D')
# 確認
watcher.head(3)

In [ ]:
# 不要な2行の削除
df = watcher.drop([0, 1])

# 列インデックスの付け直し
df.columns = ['year', 'month', 'watcher']

# '年'の除去
df.year.replace('年', '', regex=True, inplace=True)
df.head(1)

In [ ]:
# 行インデックスを年月の属性に置き換える
# 最初の年、月を取得、整数に変換
y = int(df.head(1).year)
m = int(df.head(1).month)

# 文字列に再変換 '2002-01'
start_date = '{}-{:02}'.format(y, m)

# 年月の区間を生成し、行インデックスとして指定
# - start: 開始日
# - freq: 間隔 'MS' 月間隔、月の最初の日
# - periods: 繰り返し回数
watcher_range = pd.date_range(start=start_date, freq='MS', periods=len(df))
df.index = watcher_range

# 不要な列の削除
df = df.drop(['year', 'month'], axis=1)

# 確認
df

<hr>

### 日経平均株価の読み込み

https://indexes.nikkei.co.jp/nkave/historical/nikkei_stock_average_monthly_jp.csv
- 配布資料には入っていませんので、ダウンロードして data フォルダ以下に置いてください

### 以下は説明のみ

- 実行は各自で行ってください

In [ ]:
# 読み込んで確認
# - encoding: 日本語の文字コードを指定
stock = pd.read_csv('data/nikkei_stock_average_monthly_jp.csv', encoding='shift_jis')
stock.head(2)

In [ ]:
# 末尾も確認
# - 株価以外の値が入っている
stock.tail(1)

In [ ]:
# 処理できないデータの削除
stock = stock.drop(len(stock) - 1)
# 確認
stock.tail(1)

In [ ]:
# 'データ日付' を行インデックスとして指定
stock.index = pd.to_datetime(stock['データ日付'])
# 確認
stock

In [ ]:
# 月ごとの'終値'を月の株価として、df に追加
df['stock'] = stock.loc[watcher_range]['終値']
# 確認
df

In [ ]:
# 2つのスケールが異なるグラフを重ねたいため、描画エリアのオブジェクトを取得
fig, ax = plt.subplots()
# 最初のグラフを描画
sns.lineplot(data=df.stock, ax=ax)
# 2軸目の領域を取得
ax2 = ax.twinx()
# - ax: 2軸目を指定して、2つめのグラフを描画
sns.lineplot(data=df.watcher, ax=ax2, color='orange')